# Library Circulation Data

In [ ]:
import math
import pickle

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

loans = (
    pd.read_csv("../data/Number Loans by date loaned for 2022.csv")
    .rename(
        columns={
            "Loan Date": "date",
            "Loans (Not In House)": "n_loans",
            "Permanent LC Classification Code": "lcc",
            "Patron Group": "patron",
        }
    )
    .drop(columns="Library Name")
)
loans.dropna(subset="patron", inplace=True)

loans["patron"] = loans.patron.replace("Non-Dartmouth", "Guests")

loans

In [ ]:
n_loans = loans.groupby(["date", "patron"]).n_loans.sum().reset_index(1)
n_loans = n_loans.pivot(columns=["patron"]).fillna(0)
n_loans.columns = n_loans.columns.droplevel(0)
n_loans

In [ ]:
n_loans["Interlibrary Loan"] += n_loans["BorrowDirect"]
n_loans.drop(columns=["BorrowDirect", "Delete"], inplace=True)
n_loans

In [ ]:
with open("../res/yarn_colors.pickle", "rb") as f:
    yarn_colors = pickle.load(f)

We give 0 its own color

The remaining data is then binned.

Most patron groups' data is simply binned into 10 equal bins.

The faculty group, however, has a large outlier, which would distort the data. In this case, we use 9 percentiles instead.


In [ ]:
from collections import defaultdict


def default_value():
    return "counts"


quantization_method = defaultdict(default_value)
quantization_method["Faculty"] = "percentiles"

colors = {
    "Undergraduate": [
        yarn_colors["celadon heather"],
        yarn_colors["spearmint"],
        yarn_colors["alfalfa"],
        yarn_colors["peapod"],
        yarn_colors["edamame"],
        yarn_colors["macaw"],
        yarn_colors["grass"],
        yarn_colors["forest heather"],
        yarn_colors["ivy"],
        yarn_colors["douglas fir"],
    ],
    "Graduate": [
        yarn_colors["bluebell"],
        yarn_colors["sky"],
        yarn_colors["pool"],
        yarn_colors["ciel"],
        yarn_colors["chicory"],
        yarn_colors["blue"],
        yarn_colors["delta"],
        yarn_colors["celestial"],
        yarn_colors["jay"],
        yarn_colors["navy"],
    ],
    "Faculty": [
        yarn_colors["white"],
        yarn_colors["cream"],
        yarn_colors["mist"],
        yarn_colors["finnley heather"],
        yarn_colors["gosling"],
        yarn_colors["silver"],
        yarn_colors["marble heather"],
        yarn_colors["ash"],
        yarn_colors["asphalt heather"],
        yarn_colors["black"],
    ],
    "Staff": [
        yarn_colors["almond"],
        yarn_colors["suede"],
        yarn_colors["doe"],
        yarn_colors["mongoose"],
        yarn_colors["toffee"],
        yarn_colors["brown sugar"],
        yarn_colors["hazelnut"],
        yarn_colors["bison"],
        yarn_colors["bark"],
        yarn_colors["bittersweet heather"],
    ],
    "Alum": [
        yarn_colors["pennyroyal"],
        yarn_colors["hyacinth"],
        yarn_colors["lantana"],
        yarn_colors["aster"],
        yarn_colors["french lavender"],
        yarn_colors["urchin"],
        yarn_colors["majestic"],
        yarn_colors["mulberry"],
        yarn_colors["mulberry"],
        yarn_colors["eggplant"],
    ],
    "DHMC Staff": [
        yarn_colors["custard"],
        yarn_colors["semolina"],
        yarn_colors["canary"],
        yarn_colors["turmeric"],
        yarn_colors["sweet potato"],
        yarn_colors["orange"],
        yarn_colors["masala"],
        yarn_colors["masala"],
        yarn_colors["masala"],
        yarn_colors["autumn heather"],
    ],
    "Sponsored": [
        yarn_colors["sagebrush"],
        yarn_colors["marina"],
        yarn_colors["opal heather"],
        yarn_colors["seafaring"],
        yarn_colors["teal"],
        yarn_colors["shire heather"],
        yarn_colors["rainforest heather"],
        yarn_colors["shoal"],
        yarn_colors["aurora heather"],
    ],
    "Guests": [
        yarn_colors["rose hip"],
        yarn_colors["tomato"],
        yarn_colors["serrano"],
        yarn_colors["raspberry heather"],
        yarn_colors["lipstick"],
        yarn_colors["pimento"],
        yarn_colors["hollyberry"],
        yarn_colors["lingonberry heather"],
        yarn_colors["garnet heather"],
        yarn_colors["currant"],
    ],
    "Interlibrary Loan": [
        yarn_colors["blush"],
        yarn_colors["blossom heather"],
        yarn_colors["ice lily"],
        yarn_colors["cotton candy"],
        yarn_colors["conch"],
        yarn_colors["peony"],
        yarn_colors["rouge"],
        yarn_colors["cosmopolitan"],
        yarn_colors["fuchsia"],
        yarn_colors["mauve"],
    ],
}


def quantize_data(data, mode="counts"):
    """Quantize the data by either binning or using percentiles"""

    if mode == "counts":
        # Zero checkouts get their own color, so get 9 bins for the remaining values
        bins = pd.cut(data[data != 0], bins=9).cat.categories
        # Add the Zero bin
        bins = pd.IntervalIndex.from_tuples([(-1, 0), *bins.to_tuples()])
        return bins, pd.cut(data, bins=bins, labels=False).cat.codes.to_frame()
    if mode == "percentiles":
        # Zero checkouts get their own color, so get 9 bins for the remaining values
        bins = pd.qcut(data[data != 0], q=9, duplicates="drop").cat.categories
        # Add the Zero bin
        bins = pd.IntervalIndex.from_tuples([(-1, 0), *bins.to_tuples()])
        return bins, pd.cut(data, bins=bins, labels=False).cat.codes.to_frame()


for patron in colors.keys():
    patron_data = n_loans[patron]
    idx = pd.date_range("2022-01-01", "2022-12-31")
    patron_data.index = pd.DatetimeIndex(patron_data.index)
    patron_data = patron_data.reindex(idx, fill_value=0)

    bins, quantized_data = quantize_data(patron_data, quantization_method[patron])
    print(f"{patron}")
    with open(f"../out/loans-ranges-{patron}.txt", "w") as f:
        for range in bins:
            f.write(str(range.right) + "\n")
    quantized_data.index = quantized_data.index.strftime("%Y-%m-%d")
    cmap = LinearSegmentedColormap.from_list(
        patron, colors[patron], len(colors[patron])
    )
    plt.figure(figsize=(4, 6))
    ax = sns.heatmap(quantized_data, cmap=cmap)
    colorbar = ax.collections[0].colorbar
    colorbar.set_ticks(np.linspace(0.5, 8.5, 10))
    colorbar.set_ticklabels([math.floor(bin.right) for bin in bins])

    plt.title(patron + ", " + quantization_method[patron])
    plt.xticks([])
    plt.savefig(f"../out/loans-heatmap-{patron}.svg", bbox_inches="tight")
    plt.show()
    quantized_data.to_csv(
        f"../out/loans-color_index-{patron}.txt", index=None, header=[patron]
    )